In [1]:
!pip install transformers==3.1.0

     |████████████████████████████████| 890kB 14.0MB/s 
     |████████████████████████████████| 3.0MB 36.6MB/s 
     |████████████████████████████████| 1.2MB 29.3MB/s 
     |████████████████████████████████| 901kB 27.7MB/s 


In [2]:
from transformers import DistilBertTokenizerFast
from transformers import TFDistilBertForSequenceClassification
 
import tensorflow as tf
import json
import os

In [3]:
import json
import numpy as np
def retrieve_data(fileName):
  # Opening JSON file
  f = open(fileName)
  # returns JSON object as a dictionary
  data = json.load(f)
  return data

def extract_review_and_label(data, label):
  labels = [label] * len(data)
  reviews = [d['stopwords_removal_lemmatization'] for d in data]
  return reviews, labels  

In [4]:
feature_data = retrieve_data("Feature.json")
feature_reviews, feature_labels = extract_review_and_label(feature_data, 0)

user_experience_data = retrieve_data("UserExperience.json")
user_experience_reviews, user_experience_labels = extract_review_and_label(user_experience_data, 1)

rating_data = retrieve_data("Rating.json")
rating_reviews, rating_labels = extract_review_and_label(rating_data, 2)

bug_data = retrieve_data("Bug.json")
bug_reviews, bug_labels = extract_review_and_label(bug_data, 3)

reviews = np.concatenate((feature_reviews, user_experience_reviews, rating_reviews, bug_reviews))
labels = np.concatenate((feature_labels, user_experience_labels, rating_labels, bug_labels))

randomize = np.arange(len(reviews))
np.random.shuffle(randomize)
reviews = reviews[randomize]
labels = labels[randomize]

In [7]:
import re 

def preprocess(review):

  temp = ""
  lst = []
  temp = re.sub('[^a-zA-Z]',' ',review).strip()
  temp = temp.lower()

  return temp

In [8]:
results = []
for r in reviews:
  results.append(preprocess(r))

In [9]:
import pandas as pd
df = pd.DataFrame(results, columns=["sentence"])
df["label"] = labels
df.head()

,sentence,label
0,use to love this app but its not work after ne...,3
1,love love it but take way to much space,2
2,tony sgn no password if have sure five star,2
3,excellent best one,2
4,this pretty cool app try u wont regret it,2


In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df.sentence, df.label, random_state=42, stratify=df.label)

In [11]:
training_sentences = list(X_train)
validation_sentences = list(X_test)
training_labels = list(y_train)
validation_labels = list(y_test)

In [12]:
print(len(training_sentences))
print(len(validation_sentences))

2800
934


In [13]:
pd.DataFrame(training_labels).value_counts()

2    1846
1     455
3     278
0     221
dtype: int64

In [15]:
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

In [16]:
train_encodings = tokenizer(training_sentences,
                            truncation=True,
                            padding=True)
val_encodings = tokenizer(validation_sentences,
                            truncation=True,
                            padding=True)

In [17]:
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    training_labels
))

val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    validation_labels
))

In [18]:
model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased',
                                                              num_labels=4)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_projector', 'activation_13', 'vocab_transform', 'vocab_layer_norm']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier', 'classifier', 'dropout_19']
You should probably TRAIN this model on a down-stream task to be able to use i

In [20]:
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
model.compile(optimizer=optimizer, loss=model.compute_loss, metrics=['accuracy'])
model.fit(train_dataset.shuffle(100).batch(16),
          epochs=5,
          batch_size=16,
          validation_data=val_dataset.shuffle(100).batch(16))

Epoch 1/5
Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.
175/175 [==============================] - 312s 2s/step - loss: 0.8038 - accuracy: 0.6743 - val_loss: 0.7388 - val_accuracy: 0.6906
Epoch 2/5
175/175 [==============================] - 300s 2s/step - loss: 0.6331 - accuracy: 0.7139 - val_loss: 0.7134 - val_accuracy: 0.6991
Epoch 3/5
175/175 [==============================] - 298s 2s/step - loss: 0.5205 - accuracy: 0.7639 - val_loss: 0.8587 - val_accuracy: 0.6884
Epoch 4/5
175/175 [==============================] - 298s 2s/step - loss: 0.4221 - accuracy: 0.8214 - val_loss: 0.9657 - val_accuracy: 0.6692
Epoch 5/5
175/175 [==============================] - 301s 2s/step - loss: 0.3373 - accuracy: 0.8604 - val_loss: 1.0833 - val_accuracy: 0.6724


In [21]:
model.save_pretrained("app_clf_model")

In [23]:
loaded_model = TFDistilBertForSequenceClassification.from_pretrained("app_clf_model")

Some weights of the model checkpoint at app_clf_model were not used when initializing TFDistilBertForSequenceClassification: ['dropout_19']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at app_clf_model and are newly initialized: ['dropout_39']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [24]:
test_sentence = preprocess("please add crop photo option")
predict_input = tokenizer.encode(test_sentence,
                                 truncation=True,
                                 padding=True,
                                 return_tensors="tf")

tf_output = loaded_model.predict(predict_input)[0]
print(tf_output)

[[ 2.5658114  -1.8882316  -0.87370414 -0.29608992]]


In [25]:
tf_prediction = tf.nn.softmax(tf_output, axis=1).numpy()[0]
print(tf_prediction)

[0.9083711  0.01056567 0.02914075 0.05192247]
